In [2]:
# importing required modules
import PyPDF2
import re
import pandas as pd
import os
from bs4 import BeautifulSoup
from datetime import datetime

In [3]:
name = 'Abaji anandrao jadhav cibill report.pdf'
pdfFileObj = open(name, 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj).getPage(1).extractText()
# pdfReader = pdfReader.replace('\n','')
# pdfReader = BeautifulSoup(pdfReader.content(), 'html.parser')
pdfReader

'Page 2 Copyright © 2019 Equifax Credit Information Services Private LimitedReport Order No: 606121585 Date  \t : 06-12-2022\nTime \t : 13:25:48Equifax Credit Report with Score\nAccount Details :\nAccounts\nAcct # : 077273410000022 Balance: Rs. 8,12,065 Open: Yes Date Reported: 31-08-2022\nInstitution : Bank of India Past Due Amount: 0 Interest Rate: 10.75 Date Opened: 12-06-2019\nType: Mudra Loans - Shishu / Kishor / \nTarunLast Payment: Last Payment Date: 17-06-2021 Date Closed:\nOwnership Type: Individual Write-off Amount: Sanction Amount : Rs. 9,00,000 Reason:\nRepayment Tenure: 45 Monthly Payment Amount: Rs. 19,784 Credit Limit: Collateral Value:\nDispute Code: Term Frequency: Monthly Collateral Type: Property\nAccount Status: Restructured Due to COVID19\nAsset Classification: Standard\nSuit Filed Status:\nHistory\nAccount Status:\nAsset Classification:\nSuit Filed Status:RCV\nSTD\n*\n08-22*\n*\n*\n07-22RCV\nSTD\n*\n06-22RCV\nSTD\n*\n05-22RCV\nSTD\n*\n04-22RCV\nSTD\n*\n03-22RCV\nS

In [4]:
# folder_loc = input("Enter folder location:")
folder_loc = "/Users/nilaygaitonde/Documents/E-RevBay/Automation/data"
# Create a list of all files in the folder
files = os.listdir(folder_loc)
# Create a list of all files with .pdf extension
pdf_files = [f for f in files if f.endswith('.pdf')]

In [5]:
def get_index(firstIndex,string):
    return int(firstIndex)+int(len(string))

In [6]:
def create_loan(text,on_page_2,completeDF,balance = 0):
    completeDF = {"type":[],"institution":[],"date_opened":[],"sanction_credit":[],"balance":[],"emi":[],"paid_principle":[],"open":[],"delinquecy":[]}
    countAcc = text.count("Acct # :")
    for _ in range(countAcc):
        loan = dict()
        delinquecy = False
        accountNoIndex = text.find("Acct # :")
        text = text[accountNoIndex+1:]
        openIndex = get_index(text.find('Open: '),'Open: ')
        openValue = text[openIndex:(text.find("Date Reported: "))].strip()

        dateOpenedIndex = get_index(text.find('Date Opened: '),'Date Opened: ')
        dateOpenedValue = text[dateOpenedIndex:(text.find("Type: "))].strip()
        date_object = datetime.strptime(dateOpenedValue, '%d-%m-%Y').date()
        # print(text)
        if(on_page_2):
            balanceIndex = get_index(text.find('Balance: '),'Balance: ')
            balance = text[balanceIndex:(text.find('Open:'))]
            if(balance[0] == "R"):
                balance = balance[4:]
            balance = int(balance.replace(',',''))
        # Find institution
        instiutionIndex = get_index(text.find('Institution : '),'Institution : ')
        instiutionName = text[instiutionIndex:(text.find('Past Due Amount'))-1]

        # Find type of loan
        typeIndex = get_index(text.find('Type: '),'Type: ')
        typeName = text[typeIndex:(text.find('Last Payment:'))-1]

        # TODO:::::::: FILL NAMES OF LOAN TYPES

        if(typeName not in ["Personal Loan","Business Loan","Credit Card","","Auto Loan"]):
            typeName = "4_Others"
        elif(typeName == "Credit Card"):
            typeName = "1_CreditCard"
        elif(typeName == "Personal Loan"):
            typeName = "3_PersonalLoan"
        elif(typeName == "Business Loan"):
            print(balance)
            typeName = "2_BusinessLoan"
        # Find EMI
        emiIndex = get_index(text.find('Monthly Payment Amount: '),'Monthly Payment Amount:')
        emiValue = text[emiIndex:(text.find('Credit Limit:'))-1]
        if(emiValue == ''):
            emiValue = 0
        else:
            emiValue = emiValue[4:]
            try:
                emiValue = int(emiValue.replace(',',''))
            except ValueError:
                emiValue = 0
        
        if(typeName == '1_CreditCard'):
            creditIndex = get_index(text.find('Credit Limit:'),"Credit Limit: Rs. ")
            creditLimit = text[creditIndex:text.find('Collateral Value')-1]
            sanction_credit = int(creditLimit.replace(',',''))
            print(sanction_credit)
            emiValue = balance*0.05
        else:
            sanctionIndex = get_index(text.find('Sanction Amount :'),"Sanction Amount : ")
            sanctionLimit = text[sanctionIndex:text.find('Reason:')]
            # print(sanctionLimit)
            if(sanctionLimit == ''):
                sanction_credit = 0
            else:
                sanctionLimit = sanctionLimit[4:]
                try:
                    sanction_credit = int(sanctionLimit.replace(',',''))
                except ValueError:
                    sanction_credit = 0
        AccountIndex = get_index(text.find('Account Status: '),'Account Status: ')
        AccountStatus = text[AccountIndex:text.find('Asset Classification')].strip()
        if(AccountStatus in [" ","Closed Account","Standard","Current Account"]):
            delinquecy = False
        else:
            delinquecy = True
        completeDF['balance'].append(int(balance))
        completeDF['institution'].append(instiutionName.strip())
        completeDF['type'].append(typeName.strip())
        completeDF['sanction_credit'].append(int(sanction_credit))
        completeDF['emi'].append(int(emiValue))
        completeDF['paid_principle'].append(int(sanction_credit-balance))
        completeDF['open'].append(openValue)
        completeDF['delinquecy'].append(delinquecy)
        completeDF['date_opened'].append(date_object)
    return completeDF

In [7]:
# finalDF = {"type":[],"institution":[],"sanction_credit":[],"balance":[],"emi":[],"paid_principle":[],"open":[],"delinquecy":[]}
# for file in pdf_files:
#     print("For name",file)
#     salary = input("enter salary:")
#     pdfFileObj = open(folder_loc+file, 'rb')
#     pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
#     for i in range(pdfReader.numPages):
#         pageObj = pdfReader.getPage(i).extract_text()
#         on_page_2  = True
#         balance = 0
#         if(i == 0):
#             try:
#                 pageObj.index("Account Details :")
#                 on_page_2 = False
#             except ValueError:
#                 on_page_2 = True
#         if not on_page_2:
#             balanceIndex = get_index(pageObj.find('Balance: '),'Balance: ')
#             balance = pageObj[balanceIndex:(pageObj.find('Open:'))]
#             if(balance[0] == "R"):
#                 balance = balance[4:]
#             balance = int(balance.replace(',',''))
#         newDF = create_loan(pageObj,on_page_2,finalDF,balance)
#         [finalDF["type"].append(i) for i in newDF["type"]]
#         [finalDF["institution"].append(i) for i in newDF["institution"]]
#         [finalDF["sanction_credit"].append(i) for i in newDF["sanction_credit"]]
#         [finalDF["balance"].append(i) for i in newDF["balance"]]
#         [finalDF["emi"].append(i) for i in newDF["emi"]]
#         [finalDF["paid_principle"].append(i) for i in newDF["paid_principle"]]
#         [finalDF['open'].append(i) for i in newDF["open"]]
#         [finalDF['delinquecy'].append(i) for i in newDF["delinquecy"]]
        

In [8]:
finalDF = {"type":[],"institution":[],"date_opened":[],"sanction_credit":[],"balance":[],"emi":[],"paid_principle":[],"open":[],"delinquecy":[]}
pdfFileObj = open('Abaji anandrao jadhav cibill report.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
for i in range(pdfReader.numPages):
    pageObj = pdfReader.getPage(i).extract_text()
    on_page_2  = True
    balance = 0
    if(i == 0):
        riskIndex = get_index(pageObj.find("Equifax Risk Score 3.1 "), "Equifax Risk Score 3.1 ")
        riskValue = int(pageObj[riskIndex:(pageObj.find("1. "))].strip())
        print(riskValue)
        try:
            pageObj.index("Account Details :")
            on_page_2 = False
        except ValueError:
            on_page_2 = True
    if not on_page_2:
        balanceIndex = get_index(pageObj.find('Balance: '),'Balance: ')
        balance = pageObj[balanceIndex:(pageObj.find('Open:'))]
        if(balance[0] == "R"):
            balance = balance[4:]
        balance = int(balance.replace(',',''))
    newDF = create_loan(pageObj,on_page_2,finalDF,balance)
    [finalDF["type"].append(i) for i in newDF["type"]]
    [finalDF["institution"].append(i) for i in newDF["institution"]]
    [finalDF["sanction_credit"].append(i) for i in newDF["sanction_credit"]]
    [finalDF["balance"].append(i) for i in newDF["balance"]]
    [finalDF["emi"].append(i) for i in newDF["emi"]]
    [finalDF["paid_principle"].append(i) for i in newDF["paid_principle"]]
    [finalDF['open'].append(i) for i in newDF["open"]]
    [finalDF['delinquecy'].append(i) for i in newDF["delinquecy"]]
    [finalDF['date_opened'].append(i) for i in newDF["date_opened"]]

538
378738
60000


In [9]:
data_df = pd.DataFrame.from_dict(finalDF)
data_df['open'] = data_df['open'].map({
    'Yes': True,
    'No': False
})
data_df = data_df.loc[data_df['open']==True]
data_df.drop(['open'],axis=1,inplace=True)
data_df.sort_values(by=['type'],ascending=True,inplace=True)
# balance = 0 and no delinquecy is false
# make filter for balance = 0 and delinquecy = false
data_df = data_df.loc[(data_df['balance']!=0) | (data_df['delinquecy']==True)]
data_df['paid_principle'] = data_df['paid_principle'].apply(lambda x: 0 if x<0 else x)
data_df

,type,institution,date_opened,sanction_credit,balance,emi,paid_principle,delinquecy
16,1_CreditCard,Axis Bank Ltd.,2019-08-13,60000,99927,4996,0,True
15,2_BusinessLoan,Bajaj Finance Limited,2018-12-31,516000,378738,0,137262,True
3,3_PersonalLoan,Si Creva capital Services \nPrivate Limite,2022-06-28,1426,1426,1426,0,True
0,4_Others,Bank of India,2019-06-12,900000,812065,19784,87935,True
1,4_Others,DMI Finance Private Limited,2022-06-03,8896,5560,0,3336,False
2,4_Others,ICICI BANK LIMITED,2019-10-19,0,-572,0,572,False
4,4_Others,Bank of India,2021-01-30,127300,134597,0,0,False
5,4_Others,Bank of India,2021-02-04,99000,109311,0,0,False
6,4_Others,Bank of India,2021-02-09,33700,27109,0,6591,False
8,4_Others,Bank of India,2021-06-24,86500,47871,1442,38629,False


In [10]:
data_df.groupby('type').sum()
salary = 1800000
df1=pd.DataFrame()
if(salary <= 50000):
    FOIR = salary*0.5
elif(salary > 50000 and salary <= 150000):
    FOIR = salary*0.6
else:
    FOIR = salary*0.7
delinquency = data_df['delinquecy'].sum()
if(delinquency>0):
    delinquency = True
else:
    delinquency = False
disposable = FOIR - data_df['emi'].sum()
# -------------------OUTPUT-------------------
print("FOIR: ",FOIR)
print("emi",data_df['emi'].sum())
print("Disposable: ",disposable)
print("Delinuency:",delinquency)
if(delinquency):
    print("You have delinuency in your loan and you are not eligible for loan")
else:
    if(disposable>0):
        print("You are eligible for loan")
    else:
        print("You are not eligible for loan")

FOIR:  1260000.0
emi 27648
Disposable:  1232352.0
Delinuency: True
You have delinuency in your loan and you are not eligible for loan


/var/folders/b4/9cg_tkrn3256x3hcybpxqnpw0000gn/T/ipykernel_8134/740472439.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data_df.groupby('type').sum()


In [11]:
data_df

,type,institution,date_opened,sanction_credit,balance,emi,paid_principle,delinquecy
16,1_CreditCard,Axis Bank Ltd.,2019-08-13,60000,99927,4996,0,True
15,2_BusinessLoan,Bajaj Finance Limited,2018-12-31,516000,378738,0,137262,True
3,3_PersonalLoan,Si Creva capital Services \nPrivate Limite,2022-06-28,1426,1426,1426,0,True
0,4_Others,Bank of India,2019-06-12,900000,812065,19784,87935,True
1,4_Others,DMI Finance Private Limited,2022-06-03,8896,5560,0,3336,False
2,4_Others,ICICI BANK LIMITED,2019-10-19,0,-572,0,572,False
4,4_Others,Bank of India,2021-01-30,127300,134597,0,0,False
5,4_Others,Bank of India,2021-02-04,99000,109311,0,0,False
6,4_Others,Bank of India,2021-02-09,33700,27109,0,6591,False
8,4_Others,Bank of India,2021-06-24,86500,47871,1442,38629,False


In [12]:
# remove rows where balance is 0 and delinquecy is false
data_df = data_df.loc[(data_df['balance']!=0) | (data_df['delinquecy']==True)]
data_df.shape

(12, 8)

In [13]:
# Get only filename from name
# filename = os.path.basename(file).split('.')[0]
# data_df.to_csv(f'{filename}.csv',index=False)

# LEFT TO DO:
<li>
    <ul>Remove entries where the balance and delinquecy is false</ul>
    <ul>Recommendation Engine</ul>
    <ul>Report generation</ul>
</li>

In [14]:
salary = 1800000
if(salary <= 50000):
    print('1')
    FOIR = salary*0.5
elif(salary > 50000 and salary <= 150000):
    print('2')
    FOIR = salary*0.6
else:
    print('3')
    FOIR = salary*0.7

3


In [15]:
sum_df = {
    'type': 'Total',
    'sanction_credit': data_df['sanction_credit'].sum(),
    'balance': data_df['balance'].sum(),
    'emi': data_df['emi'].sum(),
    'paid_principle': data_df['paid_principle'].sum(),
    'delinquecy': data_df['delinquecy'].sum(),
    'foir': FOIR,
    'disposable': FOIR - data_df['emi'].sum(),
}
sum_df = pd.DataFrame(sum_df,index=[0])
pivot_df = data_df.groupby('type').sum()
pivot_df = pivot_df.reset_index()
pivot_df = pivot_df.append(sum_df,ignore_index=True)
pivot_df.set_index('type',inplace=True)


/var/folders/b4/9cg_tkrn3256x3hcybpxqnpw0000gn/T/ipykernel_8134/1490884796.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pivot_df = data_df.groupby('type').sum()
/var/folders/b4/9cg_tkrn3256x3hcybpxqnpw0000gn/T/ipykernel_8134/1490884796.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pivot_df = pivot_df.append(sum_df,ignore_index=True)


In [16]:
pivot_df

,sanction_credit,balance,emi,paid_principle,delinquecy,foir,disposable
type,,,,,,,
1_CreditCard,60000,99927,4996,0,1,NaN,NaN
2_BusinessLoan,516000,378738,0,137262,1,NaN,NaN
3_PersonalLoan,1426,1426,1426,0,1,NaN,NaN
4_Others,1800396,1682291,21226,137063,3,NaN,NaN
Total,2377822,2162382,27648,274325,6,1260000.0,1232352.0


In [17]:
data_df.set_index('type',inplace=True)

In [19]:
pivot_df

,sanction_credit,balance,emi,paid_principle,delinquecy,foir,disposable
type,,,,,,,
1_CreditCard,60000,99927,4996,0,1,NaN,NaN
2_BusinessLoan,516000,378738,0,137262,1,NaN,NaN
3_PersonalLoan,1426,1426,1426,0,1,NaN,NaN
4_Others,1800396,1682291,21226,137063,3,NaN,NaN
Total,2377822,2162382,27648,274325,6,1260000.0,1232352.0


In [27]:
pivot_df.loc['Total']['disposable']


1232352.0